In [13]:
import json
import requests
from geopy.distance import geodesic


In [4]:
# Load data
with open('way_objects.json') as fp:
    way_objects = json.load(fp)

with open('broken_way_objects.json') as fp:
    broken_way_objects = json.load(fp)

keyset = list(way_objects.keys())
print("Original keys: ",len(keyset) )
print("Broken keys: ",len(broken_way_objects.keys()) )

for newkey in broken_way_objects.keys():
    original_key = newkey.split("_")[0]
    if(original_key in keyset):
        keyset.remove(original_key)
    keyset.append(newkey)

#lets combine two objects now
broken_way_objects.update(way_objects)
#lets remove keys which we do not need now
print("All keys after merging: ",len(broken_way_objects.keys()) )
new_ways = {}
for key in keyset:
    new_ways[key] = broken_way_objects[key]

print("Num of keys in final object: ",len(new_ways.keys()) )

Original keys:  219
Broken keys:  789
All keys after merging:  1008
Num of keys in final object:  789


In [7]:
def get_elevation(lat, lon):
    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract elevation from the response
        elevation = data['results'][0]['elevation']
        return elevation
    else:
        return None


In [41]:
def get_distance_elevation(way_coordinates):
    total_distance = 0  # Total distance in meters
    total_elevation_gain = 0

    # for i in range(len(way_coordinates) - 1):
        # Calculate distance between consecutive points
    point1 = way_coordinates[0]
    point2 = way_coordinates[-1]

    distance = geodesic(point1, point2).meters

    point1_elevation = 0
    point2_elevation = 0

    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={point1[0]},{point1[1]}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract elevation from the response
        point1_elevation = data['results'][0]['elevation']
    else:
        point1_elevation = 0

    api_url = f"https://api.open-elevation.com/api/v1/lookup?locations={point2[0]},{point2[1]}"
    response = requests.get(api_url)
    if response.status_code == 200:
        data = response.json()
        # Extract elevation from the response
        point2_elevation = data['results'][0]['elevation']
    else:
        point2_elevation = 0

    total_elevation_gain += point2_elevation - point1_elevation
    total_distance += distance
    return [total_distance,total_elevation_gain]

In [42]:
way_metrics = {}
print("Number of ways: {}".format(len(new_ways.keys())))
count = 1
for wayid, coordinates in new_ways.items():
    print(count)
    count += 1
    way_distance , way_elevation = get_distance_elevation(coordinates)
    if way_distance != 0 :
        way_gradient = (way_elevation / way_distance)*100
        way_metrics[wayid] = {
            "distance": way_distance,
            "gain": way_elevation,
            "gradient": way_gradient
        }
        print(way_metrics[wayid])



Number of ways: 789
1
{'distance': 158.821829747674, 'gain': 0.0, 'gradient': 0.0}
2
{'distance': 12.023167290307308, 'gain': 0.0, 'gradient': 0.0}
3
{'distance': 97.05266526102123, 'gain': 3.0, 'gradient': 3.091105217905721}
4
{'distance': 47.699641355579644, 'gain': 0.0, 'gradient': 0.0}
5
{'distance': 224.94312806486747, 'gain': 7.0, 'gradient': 3.111897687304051}
6
7
8
{'distance': 66.89155013115024, 'gain': 0.0, 'gradient': 0.0}
9
10
11
12
13
{'distance': 146.28159072257233, 'gain': -16.0, 'gradient': -10.937808319533868}
14
15
16
17
18
{'distance': 18.145803914875536, 'gain': 0.0, 'gradient': 0.0}
19
{'distance': 169.42053052067274, 'gain': 16.0, 'gradient': 9.443955789081699}
20
21
22
23
24
{'distance': 41.66167655656872, 'gain': 43.0, 'gradient': 103.21236098507482}
25
{'distance': 221.36254025715917, 'gain': 14.0, 'gradient': 6.324466634569721}
26
{'distance': 3.8760019325782085, 'gain': 0.0, 'gradient': 0.0}
27
28
29
{'distance': 265.65235518964346, 'gain': 114.0, 'gradient':

In [44]:
with open('way_metrics.json', 'w+') as fp:
    json.dump(way_metrics, fp)

In [45]:
print(len(way_metrics.keys()))
print(len(new_ways.keys()))

336
789


In [47]:
#lets verify something
#ways disregaded due to being of distance 0
discarded_ways = list(set(new_ways.keys()) - set(way_metrics.keys()))
print(len(discarded_ways))

453


In [53]:
def get_distance(way_coordinates):
    total_distance = 0  # Total distance in meters

    for i in range(len(way_coordinates) - 1):
        print(i)
        # Calculate distance between consecutive points
        point1 = way_coordinates[i]
        point2 = way_coordinates[i+1]

        distance = geodesic(point1, point2).meters
        total_distance += distance
        
    return total_distance

In [61]:
#checking distance=
count = 0
for wayid, coordinates in new_ways.items():
    if(len(coordinates) > 1):
        count +=1

print(count)


336
